In [ ]:
import pandas as pd
import numpy as np
import import_ipynb
from datetime import datetime
from data_config import COLS
import requests
TARGET='target_10_val'

In [ ]:
GROUP_NO=0 # REPLACE WITH YOUR GROUP NO

In [ ]:
# Load data
datestr=datetime.today().strftime('%d-%b-%Y')
# datestr='01-Feb-2024'
url = f'https://gmshroff.pythonanywhere.com/static/data/df_live_test_{datestr}.csv'
response = requests.get(url)
if response.status_code==200 :
    print(f'Downloading live test data for {datestr}')
    file_path=f'./data/df_live_test_{datestr}.csv'
    with open(file_path, 'wb+') as f:
        f.write(response.content)
    df_live=pd.read_csv(file_path)
    url = f'https://gmshroff.pythonanywhere.com/static/data/df_live_train_{datestr}.csv'
else: print(f'test data for {datestr} not ready')
response = requests.get(url)
if response.status_code==200:
    print(f'Downloading live training data for {datestr}')
    file_path=f'./data/df_live_train_{datestr}.csv'
    with open(file_path, 'wb+') as f:
        f.write(response.content)
    df_train=pd.read_csv(file_path)
else: print(f'train data for {datestr} not ready')

In [ ]:
class DummyModel:
    def __init__(self):
        pass
    def fit(self,X_train,y_train):
        pass
    def predict(self,X_test):
        y_preds=np.array([2 for i in range(X_test.shape[0])])
        # print(X_test,y_preds)
        return y_preds
classifier=DummyModel()

In [ ]:
### INSERT YOUR CODE TO LOAD AND ADAPT/FINE-TUNE YOUR CLASSIFIER HERE

In [ ]:
# Function to run your classifier
USE_CLASSIFIER=True # Make this True if you have prepared a classifier
def run_model(df,df_live_seq,classifier=None):
    pf=pd.DataFrame(columns=['id','predictions'])
    if USE_CLASSIFIER:
        X_train=np.array(df_live_seq[COLS].values)
        y_train=df_live_seq[TARGET].values.squeeze()
        y_train=np.array([int(y*4) for y in y_train])
        # print(X_train.shape,y_train.shape)
        classifier.fit(X_train,y_train)
        X_test=np.array(df[COLS].values)
        y_preds=classifier.predict(X_test)
        # print(y_preds)
        pf['id']=df['id']
        pf['predictions']=y_preds/4
    else:
        pf['id']=df['id']
        pf['predictions']=.5
    return pf

In [ ]:
# Run the classifier
pf=run_model(df_live,df_train,classifier)

In [110]:
pf.to_csv(f'./data/predictions_{datestr}_group{GROUP_NO}.csv',index=False)

In [ ]:
print(f"Submit the prediction file ./data/predictions_{datestr}_group{GROUP_NO}.csv via Prexys")